In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import random

import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../lib')

pd.set_option('display.max_columns', 500)

from datetime import datetime
from numpy import inf
from math import prod
from IPython.display import clear_output

from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error,mean_absolute_percentage_error
from sklearn.linear_model import LinearRegression

# Конфиг
from model_functions import *
from model_features import *
from model_metrics import *

# Для автообновления конфига
%load_ext autoreload
%autoreload 2

### 1. Подготовка

#### Загружаем данные

В качестве прогнозных макроданных используем **базовый вариант развития Банка России** (всё в темпах прироста)

Значения дамми-переменных проставим нулевыми (это легко можно поменять)

In [90]:
data=pd.read_excel('../data/data_predict.xlsx',sheet_name='data')

#### Генерим лаги

In [91]:
data_lag=pd.DataFrame()
# Сдвигаем период на 1 вперёд относительно data_delta
data_lag['period']=data['period'][1:]
# Добавляем лаги
for c in data.drop('period',axis=1).columns:
    data_lag[f'l1_{c}']=data[c][:-1].values

#### Берём лаги тех признаков, которые входят в модель

In [92]:
# Признаки с лагом
features_lag=[c for c in model_features if 'l1' in c]
# Признаки с лагом, но без префикса l1
features_lag_without_l1=[c.replace('l1_','') for c in features_lag]
# Лаговые данные
data_lag=data_lag[['period']+features_lag]

#### Объединяем

In [109]:
data_predict=pd.merge(data.drop(features_lag_without_l1,axis=1),data_lag,on='period')

### 2. Прогнозы

#### Загружаем коэффициенты линейной регрессии

In [11]:
model_coef=pd.read_excel('../results/Linreg_Results.xlsx',sheet_name='coef')[:-1]

#### Строим поквартальные прогнозы

In [116]:
# Нужный порядок столбцов
quarter_preds=data_predict[model_coef.feature]
# Пустая строка
quarter_preds.loc[len(quarter_preds)] = ['miss','miss','miss','miss','miss','miss','miss']
# Прогноз
for i in range(len(quarter_preds)-1):
    quarter_preds['l1_target'][i+1]=np.sum(model_coef['coefficient'].values*quarter_preds.iloc[i].values)
# Нужный формат
quarter_preds=pd.DataFrame({'period': data.period, 'prediction': quarter_preds['l1_target']})

#### Сохраняем

In [118]:
quarter_preds.to_excel('../results/Quarter_Predictions.xlsx',index=0)